In beginning to clean up the code, as mentioned Mary Lauren began to clean up the code initially by creating a data path to the data, and following by reading in each of the data sets given the data path and file name. From there, 

In [12]:
###
#   Mary Lauren Benton, 2017
###

###
#   imports & variables
###
import matplotlib.pyplot as plt
import pandas as pd
import numpy  as np
import csv
data_path = "/Users/sarahmaddox/Desktop/Data/" ##THIS NEEDS TO BE CHANGED BASED ON WHERE THE DATA IS


###
#   ADT
### 
adt = pd.read_csv(data_path+"FONNESBECK_ADT.csv", na_values=[''],
                  parse_dates=['Admission_date', 'Event_Date', 'DISCHARGE_DATE'],
                  encoding = "ISO-8859-1")
adt.head()

# rename the columns and replace event strings with simpler versions
# #todo -- expand categorical variables using get_dummies
adt_clean = (adt
             .rename(columns={"RUID": "patient_id", 
                              "Event":"adt_event", 
                              "Admission_date": "admission_date",
                              "Event_Date": "adt_event_date", 
                              "SRV_CODE": "srv_code",
                              "CHIEF_COMPLAINT": "chief_complaint", 
                              "DISCHARGE_DATE": "discharge_date"})
             .replace({'adt_event': {'.*Admit': 'admit',
                                     '.*Discharge': 'discharge', 
                                     '.*Transfer': 'transfer'}}, regex=True))

adt_clean.head()

# calculate the amount of missing data in the ADT table
adt_clean.isnull().sum()

# presumably only discharges will have discharge dates; these actual missing data
# #todo -- decide how to handle missing dates
df = (adt_clean[adt_clean.adt_event == 'discharge']).isnull().sum()

In [10]:
df

patient_id           0
adt_event            0
admission_date       2
adt_event_date       0
srv_code             0
chief_complaint    265
discharge_date       0
dtype: int64

In [11]:
from datetime import datetime
from datetime import date, time 
df = pd.DataFrame(adt_clean, columns = ['patient_id','adt_event','admission_date', 'discharge_date'])
x = df[df.adt_event != 'transfer']
x = x[x.adt_event != 'discharge']
# adt_final
x = x[['patient_id','admission_date', 'discharge_date']]
x.head()

x.admission_date.apply(lambda d: datetime.strptime(d, '%m/%d/%y'))

ValueError: unconverted data remains: 07

In [2]:
###
#   BMI
###
bmi = pd.read_csv(data_path+"FONNESBECK_BMI.csv", parse_dates=['Date_BMI'], infer_datetime_format=True)
bmi.head()

bmi_clean = (bmi
             .rename(columns={"RUID": "patient_id", 
                              "BMI": "bmi",
                              "Date_BMI": "bmi_date", 
                              "BMI_Weight": "weight",
                              "BMI_Height": "height", 
                              "Pregnancy_Indicator": "pregnant"}))
bmi_clean['bmi_date'] = pd.to_datetime(bmi_clean.bmi_date, errors='coerce')
bmi_clean.head()

# small amount of missingness
# #todo -- possible to fill in missing if same patient
bmi_clean.isnull().sum()

patient_id      0
bmi           307
bmi_date        9
weight        307
height          0
pregnant        0
dtype: int64

In [3]:
###
#   BP
###
bp = pd.read_csv(data_path+"FONNESBECK_BP.csv", parse_dates=['Measure_date'], infer_datetime_format=True)
bp.head()

bp_clean = (bp
            .rename(columns={"RUID": "patient_id", 
                             "SYSTOLIC": "systolic",
                             "DIASTOLIC": "diastolic", 
                             "Measure_date": "bp_date"}))
bp_clean['bp_date'] = pd.to_datetime(bp_clean.bp_date, errors='coerce')
bp_clean.head()

# only missing dates, may not need to address
bp_clean.isnull().sum()

patient_id     0
systolic       0
diastolic      0
bp_date       26
dtype: int64

In [4]:
###
#   CPT
###
cpt = pd.read_csv(data_path+"FONNESBECK_CPT.csv", parse_dates=['Event_date'], infer_datetime_format=True)
cpt.head()

cpt_clean = (cpt
             .rename(columns={"RUID": "patient_id", 
                              "CPT_Code": "cpt_code",
                              "Event_date": "cpt_event_date"}))
cpt_clean.head()

# no missing data
cpt_clean.isnull().sum()

patient_id        0
cpt_code          0
cpt_event_date    0
dtype: int64

In [5]:
###
#   EGFR
###
egfr = pd.read_csv(data_path+"FONNESBECK_EGFR.csv", parse_dates=['egfr_date'], infer_datetime_format=True)
egfr.head()

egfr_clean = (egfr
              .rename(columns={"RUID": "patient_id", 
                               "EGFR": "egfr"}))
egfr_clean.head()

# no missing data
egfr_clean.isnull().sum()

patient_id    0
egfr          0
egfr_date     0
dtype: int64

In [6]:
###
#   ICD9
###
icd9 = pd.read_csv(data_path+"FONNESBECK_ICD9.csv", parse_dates=['Event_date'], infer_datetime_format=True)
icd9.head()

icd9_clean = (icd9
              .rename(columns={"RUID": "patient_id", 
                               "ICD9_Code": "icd9_code",
                               "Event_date": "icd9_event_date"}))
icd9_clean.head()

# no missing data
icd9_clean.isnull().sum()


patient_id         0
icd9_code          1
icd9_event_date    0
dtype: int64

In [43]:
###
#   LAB
###
lab = pd.read_csv(data_path+"FONNESBECK_LAB.csv", parse_dates=['Lab_date'], 
                  infer_datetime_format=True, quoting=csv.QUOTE_NONE, na_values=['>'])
lab.head()

lab_clean = (lab
             .rename(columns={"RUID": "patient_id", 
                              "Lab_name": "short_lab_name",
                              "Lab_date": "lab_date", 
                              "Lab_value": "lab_value"}))
lab_clean.head()

# decent number of missing lab values
# may be able to impute missing values if same patient 
lab_clean.isnull().sum()

patient_id            0
short_lab_name        4
lab_date              0
lab_value         18489
dtype: int64

In [7]:
###
#   MED
###
med = pd.read_csv(data_path+"FONNESBECK_MED.csv", parse_dates=['Entry_Date'], infer_datetime_format=True)
med.head()

med_clean = (med
             .rename(columns={"RUID": "patient_id", 
                              "Entry_Date": "drug_entry_date",
                              "Drug_Name": "drug_name", 
                              "DRUG_FORM": "drug_form",
                              "DRUG_STRENGTH": "drug_strength",
                              "Route": "drug_route",
                              "Dose_Amt": "drug_dose",
                              "Drug_Freq": "drug_freq",
                              "Duration": "drug_duration"}))
med_clean.head()

# lots of missing data in this table
# many cols likely uninformative; drug_name might be most useful
med_clean.isnull().sum()

patient_id                0
drug_entry_date        1110
drug_name               201
drug_form          10178627
drug_strength       4950659
drug_route          4431974
drug_dose           8842506
drug_freq           4702238
drug_duration      12626046
dtype: int64

In [8]:

###
#   PHENOTYPE
###
phenotype = pd.read_csv(data_path+"FONNESBECK_phenotype.csv", parse_dates=['DOB', 'DOD'], infer_datetime_format=True)
phenotype.head()

phenotype_clean = (phenotype
                   .rename(columns={"RUID": "patient_id", 
                                    "Sex": "sex",
                                    "DOB": "DOB", 
                                    "DOD": "DOD",
                                    "Race": "race"})
                   .replace({'sex':  {'F': 0,'M': 1, 'U': 'NaN', '.': 'NaN', 'NA': 'NaN'}}))
phenotype_clean.head()

# lots of missing DOBs and sex
# living patients will have missing DOD
# #todo -- decide whether to collapse race into fewer categories
phenotype_clean.isnull().sum()

patient_id       0
sex             43
DOB             43
DOD           6642
race             0
dtype: int64